In [ ]:
#Import libraries
import numpy as np
import cv2
import os
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Activation
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix,classification_report,ConfusionMatrixDisplay



In [ ]:
#Load save data
data=np.load('data-memory-save-FER.npy')
target=np.load('target-memory-save-FER.npy')

In [ ]:
print(data.shape)
print(target.shape)

In [ ]:
print(data[1000])
print(target[1000])

In [ ]:
#Model

model=Sequential()

model.add(Conv2D(256,(3,3),input_shape=(150,150,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(200,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dense(5,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224,224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(5, activation="softmax")(headModel)


model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    
    layer.trainable = False

model.compile(loss="categorical_crossentropy", optimizer='adam',metrics=["accuracy"])
model.summary()


In [ ]:
img_size=150

def batcher(data,target):
    
    global img_size,batch_size
    
    def preprocess(img_name):

        img=cv2.imread(os.path.abspath(img_name))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(img_size,img_size))/255
        
        return img.reshape(img_size,img_size,1)


    while(True):
        
        data_new=[]
        target_new=[]
        inds=np.random.randint(0,len(data),batch_size)
        
        for index in inds:

            data_new.append(preprocess(data[index]))
            target_new.append(target[index])
        

        yield np.array(data_new),np.array(target_new)

In [ ]:
img_size=150

def test_batcher(data,target):
    
    global img_size,batch_size
    
    data_new=[]
    target_new=[]
    
    def preprocess(img_name):

        img=cv2.imread(os.path.abspath(img_name))
        img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(img_size,img_size))/255

        return img.reshape(img_size,img_size,1)

    for index in range(len(data)):

        data_new.append(preprocess(data[index]))
        target_new.append(target[index])
        
    return np.array(data_new),np.array(target_new)

In [ ]:
#Split the dataset
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.2)

In [ ]:
print(train_data[:3])

In [ ]:
print(test_data[:3])

In [ ]:
#Save best models
checkpoint=ModelCheckpoint(filepath='models-{epoch:02d}',monitor='val_accuracy',mode='max',save_best_only=True)

batch_size=8

#Train the model
history = model.fit(batcher(train_data,train_target),epochs=10,validation_data=batcher(test_data,test_target),
                    batch_size=batch_size,steps_per_epoch=len(train_data)//batch_size,
                    validation_steps=len(test_data)//batch_size,callbacks=[checkpoint])

model.save('model2_1.h5')

In [ ]:
#Create chart
plt.plot(history.history['loss'],'r',label='train loss')
plt.plot(history.history['val_loss'],'b',label='val loss')
plt.legend()
plt.xlabel('epoch #')
plt.ylabel('loss')

In [ ]:
#Create chart

plt.plot(history.history['accuracy'],'g',label='accuracy')
plt.plot(history.history['val_accuracy'],'b',label='val_accuracy')
plt.legend()
plt.xlabel('epoch #')
plt.ylabel('loss')

In [ ]:
# import tensorflow as tf
# model = tf.keras.models.load_model('models-13')

In [ ]:
vald_data,val_target=test_batcher(test_data,test_target)

In [ ]:
#Prediction
predicted_target=model.predict(vald_data)
matrix=confusion_matrix(np.argmax(val_target,axis=1),np.argmax(predicted_target,axis=1))

print(matrix)
print(classification_report(np.argmax(val_target,axis=1),np.argmax(predicted_target,axis=1),target_names=['Angry','Happy', 'Neutral', 'Sad', 'Surprise']))

In [ ]:
#Confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=matrix, display_labels=['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise'])
disp.plot()
